# Graphql Querry and my common questions - How to ...

Okay time for confession. It is embarassing but using and writing queries to our db for a while, I very much was relying on the custom built tools for connecting and passing the querries to graphql. So my focus was mostly on the proper format of the querry, and not much on the API. So much that when searching for 'graphql API in python' (this is what I needed right? some API for qraphql in python?!!), in the results I noticed quite a new name to me "graphene". 
https://graphene-python.org/

Well I was pretty sure that was not the name I heared before, it was shorter. 
Yes It was this 'qgl' library.  
https://pypi.org/project/gql/

now what is the difference between the two here?

>This is a GraphQL client for Python. Plays nicely with graphene, graphql-core, graphql-js and any other GraphQL implementation compatible with the spec.



okay so ... a qraphQL. client. hmm, it is different than the API, right? how?

it says that 
> Graphene-Python is a library for building GraphQL APIs in Python easily, its main goal is to provide a simple but extendable API for making developers' lives easier.

Okay so (I think) it mean that graphene is for builing a graphQL API in python (e.g as a server)

So it means that the focus should be on python gql library. and here is the right link finally! https://gql.readthedocs.io/en/v3.0.0a5/


But I guess the reason that I could be this much oblivious, was that once you set up your client, you care mostly on how to get your data from the database.
and that is what I want to pay attention to even now.

So one thing I want to do is to send some data to the database, what should I do?

## Hasura API Calls 


### Mutations

So in grphql, the querries ( mutations, and so on) are kinda generated in advanced ( by the guy who set up the API) in a genric form that would be parametrized.

for example, here I only define the "{name: {_eq: "test1"}}"
```
query MyQuery {
  poc_schema_library(limit: 100, where: {name: {_eq: "test1"}}) {
    id
    name
  }
}
```

which results in 

```
{
  "data": {
    "poc_schema_library": [
      {
        "id": "4bbd380a-1652-4c64-b08e-7cd6a612b0d9",
        "name": "test1"
      },
      {
        "id": "c59bea72-6c4d-4630-80c4-e4beba8b049a",
        "name": "test1"
      },
      {
        "id": "15c9b962-a076-4d85-a637-65f9bfd545f4",
        "name": "test1"
      },
      {
        "id": "0936db2d-ffda-455b-9ad0-12477c6f36b3",
        "name": "test1"
      }
    ]
  }
}
```

**Question:** What if I had more conditions for my querry??

**note** I need to distinguish, that these querry and mutation statements, are mostly for Hasura.
and that graphql, may allow many diiferent kinds of data requests.

okay now for pushing the data to the database I should use a "mutation".

I see three types of mutations: insert, update, and delete. now what is each?



### Simple Insert Statement

starting with the insert one, seems the simplest way to push data:

```
mutation MyMutation {
  insert_poc_schema_library(objects: {id: "1", name: "test_name", schema: "test_schema"}) {
    affected_rows
  }
}
```

which sends one object to the table "poc_schema_library". If I wanted to send more object?
it would become like this:

```
objects: [ {id: "1", name: "test_name", schema: "test_schema"}, {id: "2", name: "test_name2", schema: "test_schema2"}]
```


okay seems like an array of dictionaries. right?! 

Not quite. The keys should not have "" mark around them, e.g. 'name' and not '"name"'

So let's say we want to pass a querry like this from python to GraphQL. what should we do?




Assuming that we have  connected to the host db via gql client, for example something like this
see [here](https://pypi.org/project/gql/)
or [here](https://github.com/graphql-python/gql)
or [here](https://gql.readthedocs.io/en/v3.0.0a5/usage/variables.html)

```
url = credentials["url"]
access_key = credentials["access_key"]
gql_transport = RequestsHTTPTransport(url=url, use_json=True)
gql_transport.headers = {'x-hasura-access-key': access_key}
gql_client = Client(transport=gql_transport,fetch_schema_from_transport=True)
```

or maybe even better:


```
from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport
hasura_url = 'hsura_server_url'
hasura_secret = 'hassura_server_secret'
gql_transport = RequestsHTTPTransport(
    url=hasura_url,
    use_json=True)
gql_transport.headers = {
    'x-hasura-access-key': hasura_secret}
gql_client = Client(
    transport=gql_transport,
    fetch_schema_from_transport=True)

```

then we only need to provide the query statment to the client:

```
    result = gql_client.execute(statement)
```

(if the query has 'variables', which I get to later, there will be a variables argument to above as well)

so  basically we shall have:

```

statement = 
"""
mutation MyMutation {
  insert_poc_schema_library(objects: [ {id: "1", name: "test_name", schema: "test_schema"}, {id: "2", name: "test_name2", schema: "test_schema2"}]) {
    affected_rows
  }
}
"""
```

which is all a string. now how should we generate that kind of string?

let's say for the schema objects I have a list of records, each record a dictionary (including id, name, and schema), or maybe I get them from a dataframe, whatever, how can I put this long list in the right format?



### Insert statement with variables

Okay so for the above question, I came up with a hacky way before, but later I learned (almost) that it should be achived via variables, which I intend to figure out. But I want to start with the hacky way.
so I though that the statement is composed of two patts, one the fixed boilerplate stuff that are not going to change for a particular query type:

```

mutation MyMutation {
  insert_poc_schema_library(objects: [ list of "some objects" ]]) {
    affected_rows
  }
}
```
and then the "some objects" parts that would be different for each data we push using a query like this. 
Also the objects I have them mostly in a dict or like that. 
So I included the first part in a template, and then injected the string for the second part, to create a string for the whole statement
so for a list of records/dicts I had something like this:

```
def format_record_string(record):
    def formated_key(key):
        return "{}".format(key)

    def formated_value(value):
        if type(value) == str:
            return '"{}"'.format(value)
        else:
            return "{}".format(value)

    outstring = ",\n".join(
        "{}:{}".format(formated_key(key), formated_value(value))
        for key, value in record.items()
    )

    return "{" + outstring + "}"

records_string = ""
for record in record_list:

    records_string += format_record_string(record) + ",\n"
```

  

now obviously that is not the way to go, but I couldn't find a better way, until I heared we could use variables:

```
query2 = gql(
    '''
    mutation forecasts ($records: String!) {
        insert_forecasts (
            objects: $records,
        ) {
            affected_rows
        }
    }
'''
)



sth = {"records": [
                {
                    'provider': "test_hasura2",
                    'source': "hasura2",
                    'source_created_at': "2020-07-27 10:06:00-05:00",
                    'created_at': "2020-07-27 10:06:00-05:00",
                    'model_id': "test",
                    'venue': "test",
                   'market': "test",
                    'type': "test",
                    'product': "test",
                    'settlement_point': "test",
                    'interval_beginning_time_utc': "2020-07-27 11:00:00-05:00",
                    'interval_ending_time_utc': "2020-07-27 12:00:00-05:00",
                    'uom': "mwh",
                    'status': "Final",
                    'id': "test",
                    'probability_band_id': "test",
                    'currency': "$"
                }
            ]}


```

and then

```
gql_client.execute(query2, variable_values=sth)
```


also

```
query3 = gql(
    '''
    mutation push_forecasts ($objects: [forecasts_input!]!) {
        insert_forecasts (
            objects: $objects,
            
        ) {
            affected_rows
        }
    }
'''
)

sth = {"objects": [
                {
                    'provider': "test_hasura2",
                    'source': "hasura2",
                    'source_created_at': "2020-07-27 10:06:00-05:00",
                    'created_at': "2020-07-27 10:06:00-05:00",
                    'model_id': "test2",
                    'venue': "test",
                   'market': "test",
                    'type': "test",
                    'product': "test",
                    'settlement_point': "test",
                    'interval_beginning_time_utc': "2020-07-27 11:00:00-05:00",
                    'interval_ending_time_utc': "2020-07-27 12:00:00-05:00",
                    'uom': "mwh",
                    'status': "Final",
                    'id': "test2",
                    'probability_band_id': "test2",
                    'currency': "$"
                }
            ]}


gql_client.execute(query3   , variable_values=sth)

```
            


okay so these give kind of an idea on how it should be done, the  "sth" here that we pass as variables, is a dict, which includes all the variable defenition.
so here we only have "objects" defined as variable. 



But there is still a big question mark on the type we put in from of the variable ``` ($objects: [ orecasts_input! ]!) ```

okay so more on this variables to come...



Also In Hassura it looked to me that there are some other ways also to define the insert statement. so should look into this

## Upsert Statement - insert with constraint on conflictss

So I want to include and dive into this upsert statmenet. Basially insert tries to push new data to the table, but what if some data exactly like what we trying to push already exist? Think if we are runing some code repeatedly that push the same data. With insert the database will complain.
usually in such cases we want to update the record that already exist with the new one, oh wait a minute, didn't I say that they are the same?
if they are the same then why update it? so it does not mean necessarily that all fields are the same some fields that should be unique are repeated.
So in a table in database we are inserting records, what is the database criteria for identifying a record as redundant.
The field or fields that are supposed to be unique for records, i.e. the primary keys should not be the same.
you can use those field(s)/primar key(s) to refer to an object if existing and update it.

Oh I was supposed to look into this update statmenet separately.


```
            mutation upsert_forecasts ($objects: [forecasts_input!]!) {
                insert_forecasts (
                    objects: $objects,
                    on_conflict: {
                        constraint: price_prediction_pk,
                        update_columns: [source_created_at]
                    }
                ) {
                    affected_rows
                }
            }
```




side note: also I guess the 'forecast_input' is the type of the object we are sending in the graphgql api. but I need to get back to this.

Question: what if the database does not consider any new entry as duplicate? let's say that it assigns an id to each entry and stores as a unique record.
What if I wanted to put my own update condition based on some field.
for example in my case, the records even if the names were the same, but I knew that i didn't want to have some records repeated.
I guess this will not be considered "on_conflict".
So should I look for how we can palce constraint directly?

okay so my search for this (so far) did not bring any solution.
So seems to me that using insert/upsert one can't  check if a non-pk feature exist to update a record instead of inserting.

I sought to see how update mutation could serve for the purpose. well if we know an entry exist that some of its features match our description, then we can update the record, the way we want, or even use the delete mutation.
But if they don't exist it won't do anything.
looks like to me for such operation I would try an upadate and if not succesful, try insert.
shall look in the tuts.

### graphql code generator

For later, this might be interesting for gql backend.

https://graphql-code-generator.com/docs/getting-started/index


GraphQL Code Generator is a CLI tool that can generate TypeScript typings out of a GraphQL schema. When we develop a GraphQL backend, there would be many instances where we would find ourselves writing the same things which are already described by the GraphQL schema, only in a different format; for example: resolver signatures, MongoDB models, Angular services etc.





### Graphene

for later, this is also very interesting for generating a gql server in the backend.

https://docs.graphene-python.org/en/latest/quickstart/#introduction


Graphene is a library that provides tools to implement a GraphQL API in Python using a code-first approach.

Compare Graphene’s code-first approach to building a GraphQL API with schema-first approaches like Apollo Server (JavaScript) or Ariadne (Python). Instead of writing GraphQL Schema Definition Language (SDL), we write Python code to describe the data provided by your server.


### gql Clientr Schema validation

> If a GraphQL schema is provided, gql will validate the queries locally before sending them to the backend. If no schema is provided, gql will send the query to the backend without local validation.
You can either provide a schema yourself, or you can request gql to get the schema from the backend using introspection.

**Q** what is gql schema? how it looks like?

### Insert an object with an ARRAY field    

FROM https://hasura.io/docs/1.0/graphql/core/mutations/insert.html


To insert fields of array types, you currently have to pass them as a Postgres array literal.

```
mutation insert_author {
  insert_author (
    objects: [
      {
        name: "Ash",
        emails: "{ash@ash.com, ash123@ash.com}"
      }
    ]
  ) {
    affected_rows}
 ```

 which will add an array of two emails to the db. "emails": ["ash@ash.com", "ash123@ash.com"]"

 this can be done using variables too:

 ```
 mutation insert_author($emails: _text) {
  insert_author (
    objects: [
      {
        name: "Ash",
        emails: $emails
      }
    ]
  ) {
    affected_rows
    ```

 and variables defined as:

```
 {
  "emails": "{ash@ash.com, ash123@ash.com}"
}
```


well the email array here is still a string that is not easy to generate. 

### default value vs null value

okay so I encountered this question a while back, If I want to have a filed to be set as its default value or be null what should I do?
okay so for having the default values, or the autogenrated fileds (like ids) we would just ommit them.
for the null values, if we just don't include them and there is no deafault value, it would set as null. Or, we could explicitly set them as null.
better yet:

>If a field is nullable in the database, to set its value to null, either pass its value as null or just omit it from the input object.



## more on variables

okay so previously I saw how the variables are added to the query.

```
mutation insert_single_article($object: article_insert_input! ) {
  insert_article_one(object: $object) {
    id
    title
  }
}
```

with variables defined as:

```
{
  "object": {
    "title": "Article 1",
    "content": "Sample article content",
    "author_id": 3
  }
}
```

the question was how the type of the variables ( for example here object of type article_insert_input) are defined and where can we get them?

in this example it looks that the object variable is entered as just a dict. so what is the type article_insert_input and where it is defined?
looks like it is autogenerated by hasura(?!)


### Insert an object along with its related objects through relationships

see [here](https://hasura.io/docs/1.0/graphql/core/mutations/insert.html#insert-an-object-along-with-its-related-objects-through-relationships)


### Insert an object with a JSONB field

from 
https://hasura.io/blog/postgres-json-and-jsonb-type-support-on-graphql-41f586e47536/

https://hasura.io/docs/1.0/graphql/core/mutations/insert.html#insert-an-object-with-a-jsonb-field





>hstore, JSON and JSONB types enable storing schema-less (NoSQL) on Postgres
Hasura GraphQL Engine supports JSON/JSONB columns
JSONB operators are supported
Postgres views created using derived data from JSONB columns can be queried through GraphQL, with filter operators like where

```
mutation insert_author($address: jsonb) {
  insert_author (
    objects: [
      {
        name: "Ash",
        address: $address
      }
    ]
  ) {
    affected_rows
    returning {
      id
      name
      address
    }
  }
}

```

okay so the variables here are defined as usual. The interesting part (to me is what is returned)

```
  "returning": [
        {
          "id": 1,
          "name": "Ash",
          "address": {
            "city": "Bengaluru",
            "phone": "9090909090",
            "state": "Karnataka",
            "pincode": 560095,
            "street_address": "161, 19th Main Road, Koramangala 6th Block"
          }
        }
      ]
    }
  }
}
```

so here we have not defined an address type object. so if we enter anything under address, it will be okay.
but the returning results is just like any normal querry.

The even more interesting part is that we could even query this jsonb object, i.e ask to return objects that this jsonb part of them has a certain feature, like the state in their address be Karnataka...